In [2]:
# !pip install torch
# !pip install pillow
# !pip install torchvision
# !pip install function
import pandas as pd
from ST import style_transfer
# !wget https://isic-challenge-data.s3.amazonaws.com/2024/ISIC_2024_Training_Input.zip
# !unzip ISIC_2024_Training_Input.zip

In [3]:

real_malignant_data_df = pd.read_excel(
    'all_malignant_skin_tone_classification_type_of_cancer.xlsx',
    sheet_name='Sheet1',  # Specify the sheet name
    na_values=['NA', 'N/A', ''],  # Treat these values as missing
)

# Display the first 5 rows of the DataFrame
print("real",len(real_malignant_data_df))
all_train_df= pd.read_csv('train_dataset_with_skin_tone.csv')
print(all_train_df.head())
all_labeled_data = all_train_df.merge(real_malignant_data_df[['isic_id', 'iddx_3']], on='isic_id', how='left')
print("L",all_labeled_data.head())
all_labeled_data ['type'] = all_labeled_data['iddx_3_y'].fillna('benign')

# Drop the temporary 'idx_3' column
all_labeled_data .drop(columns=['iddx_3_y'], inplace=True)

# Display the updated DataFrame
print(len(all_labeled_data[all_labeled_data['type'] == 'benign']), len(all_labeled_data[all_labeled_data['target'] ==0]))

columns_to_keep = ['isic_id', 'target', 'type', 'skin_tone']

# Drop all columns except the ones in columns_to_keep
columns_to_drop = [col for col in all_labeled_data.columns if col not in columns_to_keep]
all_labeled_data = all_labeled_data.drop(columns=columns_to_drop)

# Save the updated DataFrame to a new CSV file
all_labeled_data.to_csv('train_labeled.csv', index=False)


real 393
        isic_id  target iddx_3 iddx_4 iddx_5 mel_mitotic_index  mel_thick_mm  \
0  ISIC_3283637       0    NaN    NaN    NaN               NaN           NaN   
1  ISIC_5518492       0    NaN    NaN    NaN               NaN           NaN   
2  ISIC_1821584       0    NaN    NaN    NaN               NaN           NaN   
3  ISIC_8102613       0    NaN    NaN    NaN               NaN           NaN   
4  ISIC_6194285       0    NaN    NaN    NaN               NaN           NaN   

   tbp_lv_dnn_lesion_confidence  Unnamed: 8  Unnamed: 9 skin_tone  
0                    100.000000         NaN         NaN   #BEA07E  
1                     99.976510         NaN         NaN   #9D7A54  
2                    100.000000         NaN         NaN   #BEA07E  
3                     53.199730         NaN         NaN   #6F503C  
4                     99.936348         NaN         NaN   #9D7A54  


/var/folders/jf/86wt6y815q55gbm9d3z6pmpm0000gn/T/ipykernel_26306/2348818565.py:9: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  all_train_df= pd.read_csv('train_dataset_with_skin_tone.csv')


L         isic_id  target iddx_3_x iddx_4 iddx_5 mel_mitotic_index  \
0  ISIC_3283637       0      NaN    NaN    NaN               NaN   
1  ISIC_5518492       0      NaN    NaN    NaN               NaN   
2  ISIC_1821584       0      NaN    NaN    NaN               NaN   
3  ISIC_8102613       0      NaN    NaN    NaN               NaN   
4  ISIC_6194285       0      NaN    NaN    NaN               NaN   

   mel_thick_mm  tbp_lv_dnn_lesion_confidence  Unnamed: 8  Unnamed: 9  \
0           NaN                    100.000000         NaN         NaN   
1           NaN                     99.976510         NaN         NaN   
2           NaN                    100.000000         NaN         NaN   
3           NaN                     53.199730         NaN         NaN   
4           NaN                     99.936348         NaN         NaN   

  skin_tone iddx_3_y  
0   #BEA07E      NaN  
1   #9D7A54      NaN  
2   #BEA07E      NaN  
3   #6F503C      NaN  
4   #9D7A54      NaN  
320533 32053

In [13]:
import pandas as pd
from ST import style_transfer
all_labeled_data= pd.read_csv('train_labeled.csv')
benign_data = all_labeled_data[all_labeled_data['target'] == 0]
cancer_data = all_labeled_data[all_labeled_data['target'] == 1]
benign_data_sample = benign_data.sample(n=5000)
print(len(cancer_data))
data_to_aug =  pd.concat([benign_data_sample, cancer_data], axis=0, ignore_index=True)

314


In [17]:
import cv2

def remove_black_bars(image_path, output_path):
    # Load the image
    image = cv2.imread(image_path)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Threshold the image to identify black regions
    _, thresh = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)

    # Find contours of the non-black regions
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Get the bounding box of the largest contour
    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest_contour)

        # Crop the image to the bounding box
        cropped_image = image[y:y+h, x:x+w]

        # Save the cropped image
        cv2.imwrite(output_path, cropped_image)
        print(f"Black bars removed. Cropped image saved to {output_path}")
    else:
        print("No black bars detected.")

# Example usage
remove_black_bars("ISIC_2024_Training_Input/ISIC_3406936.jpg", "output_image.jpg")

Black bars removed. Cropped image saved to output_image.jpg


In [19]:
for index, row in data_to_aug.iterrows():
    input_path = f"ISIC_2024_Training_Input/{row['isic_id']}.jpg"
    output_path = f"NO_BB/{row['isic_id']}.jpg"
    remove_black_bars(input_path, output_path)

Black bars removed. Cropped image saved to NO_BB/ISIC_7749053.jpg
Black bars removed. Cropped image saved to NO_BB/ISIC_1172581.jpg
Black bars removed. Cropped image saved to NO_BB/ISIC_9218210.jpg
Black bars removed. Cropped image saved to NO_BB/ISIC_8913439.jpg
Black bars removed. Cropped image saved to NO_BB/ISIC_0722364.jpg
Black bars removed. Cropped image saved to NO_BB/ISIC_6383705.jpg
Black bars removed. Cropped image saved to NO_BB/ISIC_4088646.jpg
Black bars removed. Cropped image saved to NO_BB/ISIC_5516944.jpg
Black bars removed. Cropped image saved to NO_BB/ISIC_8876484.jpg
Black bars removed. Cropped image saved to NO_BB/ISIC_0901147.jpg
Black bars removed. Cropped image saved to NO_BB/ISIC_7399602.jpg
Black bars removed. Cropped image saved to NO_BB/ISIC_4584008.jpg
Black bars removed. Cropped image saved to NO_BB/ISIC_9758497.jpg
Black bars removed. Cropped image saved to NO_BB/ISIC_9992214.jpg
Black bars removed. Cropped image saved to NO_BB/ISIC_1294750.jpg
Black bars

In [11]:
s1 = cancer_data.iloc[0]['isic_id']
s2 = cancer_data.iloc[1]['isic_id']

style_transfer("ISIC_2024_Training_Input/" + s1 + ".jpg","ISIC_2024_Training_Input/"+ s2 + ".jpg","new_test")

ISIC_2024_Training_Input/ISIC_9207822.jpg
<class 'int'>
(tensor([[[[0.5849, 0.5828, 0.5753,  ..., 0.6353, 0.6357, 0.6358],
          [0.5846, 0.5820, 0.5728,  ..., 0.6356, 0.6361, 0.6362],
          [0.5848, 0.5820, 0.5724,  ..., 0.6359, 0.6361, 0.6362],
          ...,
          [0.6648, 0.6655, 0.6665,  ..., 0.6549, 0.6546, 0.6549],
          [0.6628, 0.6629, 0.6647,  ..., 0.6534, 0.6533, 0.6535],
          [0.6623, 0.6627, 0.6640,  ..., 0.6528, 0.6529, 0.6530]],

         [[0.3877, 0.3857, 0.3767,  ..., 0.3886, 0.3890, 0.3890],
          [0.3873, 0.3846, 0.3751,  ..., 0.3889, 0.3895, 0.3894],
          [0.3876, 0.3855, 0.3748,  ..., 0.3897, 0.3900, 0.3901],
          ...,
          [0.4459, 0.4464, 0.4475,  ..., 0.3823, 0.3820, 0.3821],
          [0.4451, 0.4454, 0.4466,  ..., 0.3813, 0.3812, 0.3813],
          [0.4447, 0.4448, 0.4459,  ..., 0.3805, 0.3803, 0.3805]],

         [[0.3020, 0.2969, 0.2901,  ..., 0.2827, 0.2827, 0.2825],
          [0.3013, 0.2963, 0.2882,  ..., 0.2831, 0.

In [12]:
import itertools
stylized_data = pd.DataFrame(columns=['isic_id', 'skin_tone', 'type', 'target'])
for (idx1, row1), (idx2, row2) in itertools.combinations(data_to_aug.iterrows(), 2):
    if row1['type'] == row2['type'] and row1['skin_tone'] != row2['skin_tone']:
        s1 = row1['isic_id']
        s2 = row2['isic_id']
        if (s1 != s2):
            # Apply style transfer for both (s1, s2) and (s2, s1)
            style_transfer(f"ISIC_2024_Training_Input/{s1}.jpg", 
                        f"ISIC_2024_Training_Input/{s2}.jpg", 
                        f"new_tests")

            style_transfer(f"ISIC_2024_Training_Input/{s2}.jpg", 
                        f"ISIC_2024_Training_Input/{s1}.jpg", 
                        f"new_tests")
                        # Apply style transfer for both (s1, s2) and (s2, s1)
            stylized_id_1 = f"{s1}_stylized_{s2}"
            stylized_id_2 = f"{s2}_stylized_{s1}"

            # Add rows to the new DataFrame
            stylized_data = stylized_data.append({
                'isic_id': stylized_id_1,
                'skin_tone': row2['skin_tone'],
                'type': row1['type'],
                'target': row1['target']
            }, ignore_index=True)

            stylized_data = stylized_data.append({
                'isic_id': stylized_id_2,
                'skin_tone': row1['skin_tone'],
                'type': row2['type'],
                'target': row2['target']
            }, ignore_index=True)

ISIC_2024_Training_Input/ISIC_3406936.jpg
<class 'int'>
(tensor([[[[0.7396, 0.7396, 0.7396,  ..., 0.7396, 0.7396, 0.7396],
          [0.7396, 0.7396, 0.7396,  ..., 0.7396, 0.7396, 0.7396],
          [0.7396, 0.7396, 0.7396,  ..., 0.7396, 0.7396, 0.7396],
          ...,
          [0.8333, 0.8334, 0.8316,  ..., 0.8422, 0.8432, 0.8432],
          [0.8337, 0.8336, 0.8316,  ..., 0.8393, 0.8403, 0.8402],
          [0.8346, 0.8343, 0.8326,  ..., 0.8386, 0.8395, 0.8394]],

         [[0.5490, 0.5490, 0.5490,  ..., 0.5490, 0.5490, 0.5490],
          [0.5490, 0.5490, 0.5490,  ..., 0.5490, 0.5490, 0.5490],
          [0.5490, 0.5490, 0.5490,  ..., 0.5490, 0.5490, 0.5490],
          ...,
          [0.6191, 0.6177, 0.6154,  ..., 0.6479, 0.6480, 0.6476],
          [0.6189, 0.6175, 0.6152,  ..., 0.6456, 0.6458, 0.6454],
          [0.6180, 0.6168, 0.6143,  ..., 0.6436, 0.6438, 0.6432]],

         [[0.4378, 0.4378, 0.4378,  ..., 0.4378, 0.4378, 0.4378],
          [0.4378, 0.4378, 0.4378,  ..., 0.4378, 0.

/var/folders/jf/86wt6y815q55gbm9d3z6pmpm0000gn/T/ipykernel_26306/918923471.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stylized_data = stylized_data.append({
/var/folders/jf/86wt6y815q55gbm9d3z6pmpm0000gn/T/ipykernel_26306/918923471.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stylized_data = stylized_data.append({


ISIC_2024_Training_Input/ISIC_3406936.jpg
<class 'int'>
(tensor([[[[0.5623, 0.5623, 0.5623,  ..., 0.5623, 0.5623, 0.5623],
          [0.5623, 0.5623, 0.5623,  ..., 0.5623, 0.5623, 0.5623],
          [0.5623, 0.5623, 0.5623,  ..., 0.5623, 0.5623, 0.5623],
          ...,
          [0.7215, 0.7218, 0.7212,  ..., 0.7248, 0.7252, 0.7252],
          [0.7207, 0.7207, 0.7197,  ..., 0.7224, 0.7233, 0.7232],
          [0.7206, 0.7204, 0.7203,  ..., 0.7221, 0.7225, 0.7227]],

         [[0.3966, 0.3966, 0.3966,  ..., 0.3966, 0.3966, 0.3966],
          [0.3966, 0.3966, 0.3966,  ..., 0.3966, 0.3966, 0.3966],
          [0.3966, 0.3966, 0.3966,  ..., 0.3966, 0.3966, 0.3966],
          ...,
          [0.5330, 0.5323, 0.5313,  ..., 0.5564, 0.5568, 0.5567],
          [0.5323, 0.5315, 0.5303,  ..., 0.5548, 0.5555, 0.5555],
          [0.5314, 0.5305, 0.5297,  ..., 0.5536, 0.5541, 0.5540]],

         [[0.2730, 0.2730, 0.2730,  ..., 0.2730, 0.2730, 0.2730],
          [0.2730, 0.2730, 0.2730,  ..., 0.2730, 0.

/var/folders/jf/86wt6y815q55gbm9d3z6pmpm0000gn/T/ipykernel_26306/918923471.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stylized_data = stylized_data.append({
/var/folders/jf/86wt6y815q55gbm9d3z6pmpm0000gn/T/ipykernel_26306/918923471.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stylized_data = stylized_data.append({


ISIC_2024_Training_Input/ISIC_3406936.jpg
<class 'int'>
(tensor([[[[0.5177, 0.5177, 0.5177,  ..., 0.5177, 0.5177, 0.5177],
          [0.5177, 0.5177, 0.5177,  ..., 0.5177, 0.5177, 0.5177],
          [0.5177, 0.5177, 0.5177,  ..., 0.5177, 0.5177, 0.5177],
          ...,
          [0.6721, 0.6723, 0.6716,  ..., 0.6766, 0.6773, 0.6773],
          [0.6712, 0.6706, 0.6700,  ..., 0.6740, 0.6748, 0.6749],
          [0.6710, 0.6705, 0.6704,  ..., 0.6732, 0.6739, 0.6740]],

         [[0.3757, 0.3757, 0.3757,  ..., 0.3757, 0.3757, 0.3757],
          [0.3757, 0.3757, 0.3757,  ..., 0.3757, 0.3757, 0.3757],
          [0.3757, 0.3757, 0.3757,  ..., 0.3757, 0.3757, 0.3757],
          ...,
          [0.4926, 0.4914, 0.4897,  ..., 0.5179, 0.5185, 0.5183],
          [0.4919, 0.4905, 0.4888,  ..., 0.5160, 0.5169, 0.5166],
          [0.4913, 0.4899, 0.4883,  ..., 0.5142, 0.5151, 0.5149]],

         [[0.2960, 0.2960, 0.2960,  ..., 0.2960, 0.2960, 0.2960],
          [0.2960, 0.2960, 0.2960,  ..., 0.2960, 0.

...

/var/folders/jf/86wt6y815q55gbm9d3z6pmpm0000gn/T/ipykernel_26306/918923471.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stylized_data = stylized_data.append({
/var/folders/jf/86wt6y815q55gbm9d3z6pmpm0000gn/T/ipykernel_26306/918923471.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stylized_data = stylized_data.append({


ISIC_2024_Training_Input/ISIC_3406936.jpg
<class 'int'>
(tensor([[[[0.6295, 0.6295, 0.6295,  ..., 0.6295, 0.6295, 0.6295],
          [0.6295, 0.6295, 0.6295,  ..., 0.6295, 0.6295, 0.6295],
          [0.6295, 0.6295, 0.6295,  ..., 0.6295, 0.6295, 0.6295],
          ...,
          [0.7179, 0.7160, 0.7103,  ..., 0.7239, 0.7240, 0.7251],
          [0.7163, 0.7136, 0.7082,  ..., 0.7209, 0.7210, 0.7222],
          [0.7159, 0.7138, 0.7080,  ..., 0.7200, 0.7201, 0.7213]],

         [[0.4082, 0.4082, 0.4082,  ..., 0.4082, 0.4082, 0.4082],
          [0.4082, 0.4082, 0.4082,  ..., 0.4082, 0.4082, 0.4082],
          [0.4082, 0.4082, 0.4082,  ..., 0.4082, 0.4082, 0.4082],
          ...,
          [0.4815, 0.4781, 0.4709,  ..., 0.5038, 0.5042, 0.5042],
          [0.4797, 0.4763, 0.4689,  ..., 0.5014, 0.5018, 0.5017],
          [0.4783, 0.4747, 0.4673,  ..., 0.4992, 0.4996, 0.4995]],

         [[0.2971, 0.2971, 0.2971,  ..., 0.2971, 0.2971, 0.2971],
          [0.2971, 0.2971, 0.2971,  ..., 0.2971, 0.

/var/folders/jf/86wt6y815q55gbm9d3z6pmpm0000gn/T/ipykernel_26306/918923471.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stylized_data = stylized_data.append({
/var/folders/jf/86wt6y815q55gbm9d3z6pmpm0000gn/T/ipykernel_26306/918923471.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stylized_data = stylized_data.append({


ISIC_2024_Training_Input/ISIC_3406936.jpg
<class 'int'>


KeyboardInterrupt: 

In [20]:
cancer_sample

,isic_id,target,skin_tone,type
6970,ISIC_9060519,1,#E5C8A6,Basal cell carcinoma
44856,ISIC_4363101,1,#BEA07E,Basal cell carcinoma
259199,ISIC_7989726,1,#81654F,Melanoma in situ
263931,ISIC_7745168,1,#BEA07E,Melanoma in situ
163508,ISIC_5077618,1,#9D7A54,Basal cell carcinoma


In [21]:
stylized_data

,isic_id,skin_tone,type,target
0,ISIC_9060519_stylized_ISIC_4363101,#BEA07E,Basal cell carcinoma,1
1,ISIC_4363101_stylized_ISIC_9060519,#E5C8A6,Basal cell carcinoma,1
2,ISIC_9060519_stylized_ISIC_5077618,#9D7A54,Basal cell carcinoma,1
3,ISIC_5077618_stylized_ISIC_9060519,#E5C8A6,Basal cell carcinoma,1
4,ISIC_4363101_stylized_ISIC_5077618,#9D7A54,Basal cell carcinoma,1
5,ISIC_5077618_stylized_ISIC_4363101,#BEA07E,Basal cell carcinoma,1
6,ISIC_7989726_stylized_ISIC_7745168,#BEA07E,Melanoma in situ,1
7,ISIC_7745168_stylized_ISIC_7989726,#81654F,Melanoma in situ,1
